In [1]:
import pandas as pd

# Load the Excel file
file_path = "Search_collection_id_11904_schema_LegalEntity.xlsx"
xls = pd.ExcelFile(file_path)

# List all sheet names
print("Sheets in the Excel file:", xls.sheet_names)

# Assuming the main data is in the first sheet
df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])

# Show the first few rows
print(df.head())

# Show column headers
print("\nColumns in the dataset:")
print(df.columns.tolist())

# Optional: basic info
print("\nDataset info:")
print(df.info())

Sheets in the Excel file: ['Legal entities']
                                                  ID  \
0  011d978ff27cf2abb1a7523c2205b7a9d600a81e.82720...   
1  0130467638a7c16072d072905c5be355f7533a17.69386...   
2  017dacb3d008b8a11cac1e42e9fccdf61013d833.58a14...   
3  01920ad08a43892768daa3cc8ae757c3f6520fe7.4770e...   
4  02456fc2696627bbd0c1b380365b895dabab59a1.d6ae7...   

                                                 url  \
0  https://aleph.occrp.org/entities/011d978ff27cf...   
1  https://aleph.occrp.org/entities/0130467638a7c...   
2  https://aleph.occrp.org/entities/017dacb3d008b...   
3  https://aleph.occrp.org/entities/01920ad08a438...   
4  https://aleph.occrp.org/entities/02456fc269662...   

                           collection                   Name  E-Mail  Phone  \
0  Swedish Aircraft Registry - Owners   Tågalycke Konsult AB     NaN    NaN   
1  Swedish Aircraft Registry - Owners       Fly Czech s.r.o.     NaN    NaN   
2  Swedish Aircraft Registry - Owners  Swetr

In [2]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Load Excel file
file_path = "Search_collection_id_11904_schema_LegalEntity.xlsx"
df = pd.read_excel(file_path, sheet_name="Legal entities")

# Keep only relevant columns
df = df[['Name', 'Address']].copy()

# Just take first 10 rows for testing
df = df.head(10)

# Initialize geocoder with a longer timeout
geolocator = Nominatim(user_agent="swedish_aircraft_mapper", timeout=10)

# Use RateLimiter to avoid overloading server
geocode = RateLimiter(
    geolocator.geocode,
    min_delay_seconds=1,    # wait at least 1s between calls
    max_retries=2,          # retry up to 2 times
    error_wait_seconds=5,   # wait 5s after error before retry
    swallow_exceptions=True # don't crash, just return None
)

# Apply geocoding
df['location'] = df['Address'].apply(geocode)
df['latitude'] = df['location'].apply(lambda loc: loc.latitude if loc else None)
df['longitude'] = df['location'].apply(lambda loc: loc.longitude if loc else None)

# Check results
print(df)

# Optional: save test output
df.to_csv("swedish_aircraft_geocoded_test.csv", index=False)

                                    Name                Address  \
0                   Tågalycke Konsult AB                KLIPPAN   
1                       Fly Czech s.r.o.                Praha 6   
2                  Swetrail Transport AB                VÄRNAMO   
3                            AVI-LINK AS             Jonsvatnet   
4                      Kjell A Østnes AS                   OSLO   
5                          Swedewings AB             SKELLEFTEÅ   
6                  Swedair Management AB              Jönköping   
7                                 HOZ AB                   SALA   
8  Celestial Aviation Trading 26 Limited  Shannon, County Clare   
9                      UBI Leasing S.p.A                BRESCIA   

                                            location   latitude   longitude  
0  (Klippans kommun, Skåne län, Sverige, (56.15, ...  56.150000   13.166667  
1  (Praha 6, 102/19, Kafkova, Dejvice, Praha, obv...  50.098769   14.396196  
2  (Värnamo, Värnamo kommun,

In [6]:
len(df)

1992

In [ ]:
import pandas as pd
import googlemaps
import time

# Load API key from file
with open("google_api_key.txt", "r") as f:
    API_KEY = f.read().strip()

# Initialize Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

# Load Excel file
file_path = "Search_collection_id_11904_schema_LegalEntity.xlsx"
df = pd.read_excel(file_path, sheet_name="Legal entities")
df = df[['Name', 'Address']].copy()

# Function to geocode with retries
def geocode_address(address, retries=3, delay=1):
    for i in range(retries):
        try:
            if not isinstance(address, str) or not address.strip():
                return None, None
            result = gmaps.geocode(address)
            if result:
                loc = result[0]['geometry']['location']
                return loc['lat'], loc['lng']
            return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying...")
            time.sleep(delay)
    return None, None

# Limit to first 20 for safety test (remove .head(20) to run on full dataset)
num_samples = 20
sample = df.head(num_samples).copy()

# Geocode and store results
sample[['latitude', 'longitude']] = sample['Address'].apply(
    lambda addr: pd.Series(geocode_address(addr))
)

# Save results
output_file = f"swedish_aircraft_geocoded_google_num_samples={num_samples}.csv"
sample.to_csv(output_file, index=False)

print(f"Saved geocoded data to {output_file}")
print(sample.head())

In [8]:
import pandas as pd
import googlemaps
import time
from tqdm import tqdm  # progress bar

# Load API key from file
with open("google_api_key.txt", "r") as f:
    API_KEY = f.read().strip()

# Initialize Google Maps client
gmaps = googlemaps.Client(key=API_KEY)

# Load Excel file
file_path = "Search_collection_id_11904_schema_LegalEntity.xlsx"
df = pd.read_excel(file_path, sheet_name="Legal entities")
df = df[['Name', 'Address']].copy()

# Function to geocode with retries
def geocode_address(address, retries=3, delay=1):
    for i in range(retries):
        try:
            if not isinstance(address, str) or not address.strip():
                return None, None
            result = gmaps.geocode(address)
            if result:
                loc = result[0]['geometry']['location']
                return loc['lat'], loc['lng']
            return None, None
        except Exception as e:
            print(f"Error geocoding {address}: {e}, retrying...")
            time.sleep(delay)
    return None, None

# Prepare results
latitudes = []
longitudes = []

# Loop with tqdm progress bar
for addr in tqdm(df['Address'], desc="Geocoding addresses"):
    lat, lng = geocode_address(addr)
    latitudes.append(lat)
    longitudes.append(lng)
    time.sleep(0.2)  # polite delay for quota (adjust as needed)

df['latitude'] = latitudes
df['longitude'] = longitudes

# Save results
output_file = f"swedish_aircraft_geocoded_google_full.csv"
df.to_csv(output_file, index=False)

print(f"✅ Saved geocoded data to {output_file}")

Geocoding addresses: 100%|█████████████████████████████████████████████| 1992/1992 [08:33<00:00,  3.88it/s]

✅ Saved geocoded data to swedish_aircraft_geocoded_google_full.csv


In [18]:
import pandas as pd
import gmplot

# Load geocoded data
df = pd.read_csv("swedish_aircraft_geocoded_google_full.csv")

# Drop rows with missing coordinates
df = df.dropna(subset=['latitude', 'longitude'])

# Center map on Sweden (approximate centroid)
lat_center = df['latitude'].mean()
lng_center = df['longitude'].mean()

# Initialize GoogleMapPlotter
gmap = gmplot.GoogleMapPlotter(lat_center, lng_center, 6, apikey=open("google_api_key.txt").read().strip())

# Scatter points
gmap.scatter(df['latitude'], df['longitude'], color='red', size=40, marker=True)

# Save HTML
gmap.draw("aircraft_entities_map.html")
print("Saved interactive map as aircraft_entities_map.html")

Saved interactive map as aircraft_entities_map.html


In [21]:
import folium
import pandas as pd

df = pd.read_csv("swedish_aircraft_geocoded_google_full.csv").dropna(subset=['latitude','longitude'])
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=6)

for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=row['Name'],
        tooltip=row['Name']
    ).add_to(m)

m.save("aircraft_folium_map.html")

In [19]:
print(df[['latitude','longitude']].head(20))
print(df[['latitude','longitude']].dtypes)

     latitude  longitude
0   56.134900  13.129041
1   50.075538  14.437800
2   57.183160  14.047821
3   63.365431  10.588992
4   59.913869  10.752245
5   64.750244  20.950917
6   57.782614  14.161788
8   52.711782  -8.878336
9   45.541553  10.211802
10  55.541889  13.953884
11  51.507218  -0.127586
12  65.584819  22.156703
13  34.210389 -77.886812
14  59.275263  15.213411
15  59.858564  17.638927
16  59.371249  16.509805
17  55.751754  13.365501
18  59.332704  18.065625
19  58.587745  16.192421
20  59.278744  18.311052
latitude     float64
longitude    float64
dtype: object


In [20]:
len(df)

1977